In [2]:
from config.es import es
from config.settings import settings
from models.annoq_model import AnnoqData
from itertools import islice

In [46]:
async def get_annotations():
    resp = await es.search(
          index=settings.ES_INDEX,
          query={"match_all": {}},
          size=20,
    )

    results = [AnnoqData(id=hit['_id'], 
                         chr = hit['_source']['chr'],
                         pos = hit['_source']['pos'],
                         ref = hit['_source']['ref'],
                         alt = hit['_source']['alt'],
                         ANNOVAR_ensembl_Effect = hit['_source']['ANNOVAR_ensembl_Effect'],
                         ANNOVAR_ensembl_Transcript_ID = hit['_source'].get('ANNOVAR_ensembl_Transcript_ID', None),
                         ANNOVAR_ensembl_Gene_ID = hit['_source'].get('ANNOVAR_ensembl_Gene_ID', None),
                         ANNOVAR_ensembl_summary = hit['_source']['ANNOVAR_ensembl_summary'],
                         SnpEff_ensembl_Effect = hit['_source']['SnpEff_ensembl_Effect'],
                         SnpEff_ensembl_Effect_impact = hit['_source']['SnpEff_ensembl_Effect_impact']) 
              for hit in resp['hits']['hits']]
        
    return results

results = await get_annotations()
results

[AnnoqData(id='2:10662G>C', chr='2', pos=10662, ref='G', alt='C', ANNOVAR_ensembl_Effect='intergenic', ANNOVAR_ensembl_Transcript_ID=None, ANNOVAR_ensembl_Gene_ID=None, ANNOVAR_ensembl_summary='(0):intergenic(1)', SnpEff_ensembl_Effect='intergenic_region', SnpEff_ensembl_Effect_impact='MODIFIER'),
 AnnoqData(id='6:89961G>A', chr='6', pos=89961, ref='G', alt='A', ANNOVAR_ensembl_Effect='intergenic', ANNOVAR_ensembl_Transcript_ID=None, ANNOVAR_ensembl_Gene_ID=None, ANNOVAR_ensembl_summary='(0):intergenic(1)', SnpEff_ensembl_Effect='intergenic_region', SnpEff_ensembl_Effect_impact='MODIFIER'),
 AnnoqData(id='6:147651A>C', chr='6', pos=147651, ref='A', alt='C', ANNOVAR_ensembl_Effect='upstream', ANNOVAR_ensembl_Transcript_ID='ENST00000406017|ENST00000436899', ANNOVAR_ensembl_Gene_ID='ENSG00000217929|ENSG00000170590', ANNOVAR_ensembl_summary='ENSG00000170590(1):upstream(1)|ENSG00000217929(1):upstream(1)', SnpEff_ensembl_Effect='intergenic_region|upstream_gene_variant', SnpEff_ensembl_Effect

In [5]:
async def search_by_ID(id:str):

    resp = await es.search(
          index = settings.ES_INDEX,
          source = ['chr', 'pos'],
          from_= 1,
          size = 10,
          query = {
              "bool": {
                    "must": [
                      {"term": {"rs_dbSNP151": id}},
                    ]
              }
          }
    )
    return resp

results = await search_by_ID("rs189126619")
results

ObjectApiResponse({'took': 3, 'timed_out': False, '_shards': {'total': 5, 'successful': 5, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 1, 'relation': 'eq'}, 'max_score': 2.4277482, 'hits': []}})

In [4]:

async def get_mapping():
    mapping = await es.indices.get_mapping(index=settings.ES_INDEX)
    print(mapping)

x = await get_mapping()
x

{'es_index': {'mappings': {'properties': {'1000Gp3_AC': {'type': 'long'}, '1000Gp3_AF': {'type': 'float'}, '1000Gp3_AFR_AC': {'type': 'long'}, '1000Gp3_AFR_AF': {'type': 'float'}, '1000Gp3_AMR_AC': {'type': 'long'}, '1000Gp3_AMR_AF': {'type': 'float'}, '1000Gp3_EAS_AC': {'type': 'long'}, '1000Gp3_EAS_AF': {'type': 'float'}, '1000Gp3_EUR_AC': {'type': 'long'}, '1000Gp3_EUR_AF': {'type': 'float'}, '1000Gp3_SAS_AC': {'type': 'long'}, '1000Gp3_SAS_AF': {'type': 'float'}, 'ALSPAC_AC': {'type': 'long'}, 'ALSPAC_AF': {'type': 'float'}, 'ALSPAC_AN': {'type': 'long'}, 'ANNOVAR_ensembl_Closest_gene(intergenic_only)': {'type': 'text', 'fields': {'keyword': {'type': 'keyword', 'ignore_above': 10000}}}, 'ANNOVAR_ensembl_Effect': {'type': 'text', 'fields': {'keyword': {'type': 'keyword', 'ignore_above': 10000}}}, 'ANNOVAR_ensembl_Exon_Rank': {'type': 'text', 'fields': {'keyword': {'type': 'keyword', 'ignore_above': 10000}}}, 'ANNOVAR_ensembl_GO_biological_process_complete_list_id': {'type': 'text', 

In [9]:
async def get_aggregates():
    resp = await es.search(
          index = settings.ES_INDEX,
          query = {
              "bool": {
                    "must": [
                      {"term": {"chr": 2}}
                    ]
              }
          },
         aggs = {
            "gene_positions": {
                "pos_max": {
                    {"max": {"field": "pos"}}
                }
            }
         }

         )
    print(resp['aggregations'])
    return resp['aggregations']

results = await get_aggregates()
results

TypeError: unhashable type: 'dict'